In [1]:
%%capture
%load_ext autoreload
%autoreload 2

# We can also put these magic commands in `.vscode/settings.json` like this:
# "jupyter.runStartupCommands": [
#     "%load_ext autoreload",
#     "%autoreload 2"
# ]

In [2]:
from pathlib import Path

import polars as pl

from cmipiu.features import aggregate_pq_files, aggregate_pq_files_v2

print("All imports loaded successfully")

All imports loaded successfully


In [17]:
DATA_DIR = Path("../input/child-mind-institute-problematic-internet-use/")
files = [file for file in (DATA_DIR / 'series_train.parquet').iterdir()]

# cols = ["X", "Y", "Z", "enmo", "light", "battery_voltage"]
# train_agg = aggregate_pq_files(files, cols)
train_agg = aggregate_pq_files_v2(files)
train_agg.head()

Aggregating pq files: 100%|██████████| 996/996 [00:17<00:00, 56.17it/s]


id,relative_start_date_PCIAT,total_days,rolling_std_anglez_abs_std,X_mean,X_std,Y_mean,Y_std,angleZ_mean,angleZ_std,daily_avg_enmo_min,daily_avg_enmo_mean,daily_avg_enmo_std,daily_avg_enmo_max,daily_avg_light_min,daily_avg_light_mean,daily_avg_light_std,daily_avg_light_max,first7_avg_enmo_min,first7_avg_enmo_mean,first7_avg_enmo_std,first7_avg_enmo_max,first7_avg_light_min,first7_avg_light_mean,first7_avg_light_std,first7_avg_light_max
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""cefdb7fe""",13.0,24.0,9.303995,0.047879,0.523529,0.003235,0.441043,-18.724211,49.600792,0.007315,0.023521,0.012307,0.055307,5.66291,67.586945,59.117203,277.175049,null,null,null,null,null,null,null,null
"""ca33a5e7""",55.0,31.0,5.330363,0.011678,0.169036,0.114462,0.434995,-62.595814,41.026581,1.2589e-7,0.002589,0.005216,0.021023,0.636399,25.029976,36.072395,174.563477,null,null,null,null,null,null,null,null
"""0d01bbf2""",28.0,35.0,3.646807,-0.478971,0.429476,-0.037643,0.518888,-14.676959,28.848204,0.022115,0.058052,0.017665,0.099609,1.581965,39.541206,35.889221,148.635071,null,null,null,null,null,null,null,null
"""58391429""",25.0,22.0,8.10817,-0.088864,0.300414,0.045145,0.371415,-21.555964,67.857338,3.9179e-7,0.004911,0.005921,0.020968,0.0,10.284554,13.354685,58.878628,null,null,null,null,null,null,null,null
"""6b6467f4""",7.0,1.0,5.879189,-0.42547,0.471329,0.056812,0.513878,-20.495251,26.673159,0.009792,0.032944,0.032742,0.056096,8.0,9.360171,1.923572,10.720342,null,null,null,null,null,null,null,null


In [18]:
files = [file for file in (DATA_DIR / 'series_test.parquet').iterdir()]

# cols = ["X", "Y", "Z", "enmo", "light", "battery_voltage"]
# test_agg = aggregate_pq_files(files, cols)
test_agg = aggregate_pq_files_v2(files)
test_agg.head()

Aggregating pq files:   0%|          | 0/2 [00:00<?, ?it/s]

Aggregating pq files: 100%|██████████| 2/2 [00:00<00:00, 109.08it/s]


id,relative_start_date_PCIAT,total_days,rolling_std_anglez_abs_std,X_mean,X_std,Y_mean,Y_std,angleZ_mean,angleZ_std,daily_avg_enmo_min,daily_avg_enmo_mean,daily_avg_enmo_std,daily_avg_enmo_max,daily_avg_light_min,daily_avg_light_mean,daily_avg_light_std,daily_avg_light_max,first7_avg_enmo_min,first7_avg_enmo_mean,first7_avg_enmo_std,first7_avg_enmo_max,first7_avg_light_min,first7_avg_light_mean,first7_avg_light_std,first7_avg_light_max
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""00115b9f""",41.0,44.0,6.143994,-0.316383,0.453665,0.016009,0.502702,-10.580381,42.94717,0.000003,0.029791,0.025196,0.097723,1.239798,16.674427,26.515331,118.305832,0.008188,0.054789,0.029895,0.097723,1.949013,47.063744,42.372292,118.305847
"""001f3379""",68.0,23.0,8.820418,-0.004272,0.351545,0.016859,0.303812,-55.614735,50.303635,0.005051,0.012142,0.005382,0.031802,0.385574,17.674879,20.731668,84.181152,null,null,null,null,null,null,null,null


In [19]:
train = pl.read_csv(DATA_DIR / "train.csv")
test = pl.read_csv(DATA_DIR / "test.csv")

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (3960, 82)
Test shape: (20, 59)


In [20]:
train = train.join(train_agg, how='left', on='id')
test = test.join(test_agg, how='left', on='id')

print(f"New train shape after joining aggregates: {train.shape}")
print(f"New test shape after joining aggregates: {test.shape}")

New train shape after joining aggregates: (3960, 107)
New test shape after joining aggregates: (20, 84)


In [21]:
train_filepath = Path('../input/processed/train_combined_v2.csv')
train_filepath.parent.mkdir(parents=True, exist_ok=True)
train.write_csv(train_filepath)
print(f"Preprocessed training data saved in {train_filepath}")

test_filepath = Path('../input/processed/test_combined_v2.csv')
test_filepath.parent.mkdir(parents=True, exist_ok=True)
test.write_csv(test_filepath)
print(f"Preprocessed testing data saved in {test_filepath}")

Preprocessed training data saved in ../input/processed/train_combined_v2.csv
Preprocessed testing data saved in ../input/processed/test_combined_v2.csv
